Download and unzip the Monarch Graph and Phenio semantic SQL datbabase

In [1]:
!wget -q -nc https://data.monarchinitiative.org/monarch-kg/latest/monarch-kg.duckdb.gz
!gunzip -k monarch-kg.duckdb.gz  # Use -k to keep the original .gz file
!wget -q -nc https://data.monarchinitiative.org/monarch-kg/latest/phenio.db.gz  # consider hpo.db.gz for speed
!gunzip -k phenio.db.gz  # Use -k to keep the original .gz file
!mkdir -p output  # Use -p to avoid error if the directory already exists

--2024-11-11 19:12:35--  https://data.monarchinitiative.org/monarch-kg/latest/monarch-kg.duckdb.gz
Resolving data.monarchinitiative.org (data.monarchinitiative.org)... 35.208.191.193
Connecting to data.monarchinitiative.org (data.monarchinitiative.org)|35.208.191.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1628735176 (1.5G) [application/octet-stream]
Saving to: ‘monarch-kg.duckdb.gz’

monarch-kg.duckdb.g 100%[===================>]   1.52G  29.2MB/s    in 55s     

2024-11-11 19:13:30 (28.5 MB/s) - ‘monarch-kg.duckdb.gz’ saved [1628735176/1628735176]

--2024-11-11 19:13:43--  https://data.monarchinitiative.org/monarch-kg/latest/phenio.db.gz
Resolving data.monarchinitiative.org (data.monarchinitiative.org)... 35.208.191.193
Connecting to data.monarchinitiative.org (data.monarchinitiative.org)|35.208.191.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2189141060 (2.0G) [application/octet-stream]
Saving to: ‘phenio.db.gz’

phe

In [2]:
import duckdb

db = duckdb.connect('monarch-kg.duckdb', read_only=True)
db.sql("""
copy (
  select 
       subject as disease, 
       list(object) as phenotype, 
       primary_knowledge_source as source
    from edges
  where predicate = 'biolink:has_phenotype' 
       and negated <> True
       and primary_knowledge_source in ('infores:orphanet','infores:omim')
       and subject in (
          select distinct subject
            from denormalized_edges
            where subject_namespace = 'MONDO'
              and predicate = 'biolink:has_phenotype'
              and primary_knowledge_source in ('infores:orphanet','infores:omim')
            group by all
            having count(distinct primary_knowledge_source) > 1
       )     
    group by all
) to 'output/phenotype_profiles_by_source.parquet';       
""") 

In [3]:
# load the phenotype profiles into a pandas dataframe 
import pandas as pd
phenotype_profiles = pd.read_parquet('output/phenotype_profiles_by_source.parquet')
phenotype_profiles.head()

,disease,phenotype,source
0,MONDO:0008843,"[HP:0000407, HP:0001327, HP:0001677, HP:000190...",infores:orphanet
1,MONDO:0012658,"[HP:0001773, HP:0001817, HP:0001831, HP:000185...",infores:orphanet
2,MONDO:0007215,"[HP:0001204, HP:0001230, HP:0001762, HP:000177...",infores:orphanet
3,MONDO:0012072,"[HP:0000147, HP:0000819, HP:0000822, HP:000084...",infores:orphanet
4,MONDO:0010392,"[HP:0000750, HP:0001249, HP:0001251, HP:000126...",infores:orphanet


Initialize semsimian from phenio.db

In [4]:
from semsimian import Semsimian

predicates= [
    "rdfs:subClassOf",
    "BFO:0000050",
    "UPHENO:0000001",
]

semsimian = Semsimian(
    spo=None,
    predicates=predicates,
    pairwise_similarity_attributes=None,
    resource_path="./phenio.db",
)

Using Semsimian, generate termset_pairwise_similarity scores for OMIM vs Orphanet, OMIM vs OMIM and Orphanet vs Orphanet using each metric. 

In [5]:
results = []
for disease in phenotype_profiles['disease'].unique():
    result = {
        'disease': disease
    }
    # get the omim profile
    try:
        omim_profile = list(phenotype_profiles[(phenotype_profiles['disease'] == disease) & (phenotype_profiles['source'] == 'infores:omim')]['phenotype'])[0]
    except IndexError:
        print(f'No OMIM profile for {disease}, skipping')
        continue
    # get the orphanet profile
    try:
        orphanet_profile = list(phenotype_profiles[(phenotype_profiles['disease'] == disease) & (phenotype_profiles['source'] == 'infores:orphanet')]['phenotype'])[0]
    except IndexError:
        print(f'No Orphanet profile for {disease}, skipping')
        continue

    for metric in ['ancestor_information_content', 'jaccard_similarity', 'phenodigm_score']:                
        result[metric] = pairwise_similarity_result = semsimian.termset_pairwise_similarity(
            set(omim_profile), set(orphanet_profile), metric
        )
        result[f"omim_self_{metric}"] = semsimian.termset_pairwise_similarity(
            set(omim_profile), set(omim_profile), metric
        )
        result[f"orphanet_self_{metric}"] = semsimian.termset_pairwise_similarity(
            set(orphanet_profile), set(orphanet_profile), metric
        )

    results.append(result)

In [6]:
score_df = pd.DataFrame(
    [{
        'disease': r['disease'],
        'ancestor_information_content': r['ancestor_information_content']['average_score'],
        'ancestor_information_content_omim_self': r['omim_self_ancestor_information_content']['average_score'],
        'ancestor_information_content_orphanet_self': r['orphanet_self_ancestor_information_content']['average_score'],   
        'jaccard_similarity': r['jaccard_similarity']['average_score'],
        'jaccard_similarity_omim_self': r['omim_self_jaccard_similarity']['average_score'],
        'jaccard_similarity_orphanet_self': r['orphanet_self_jaccard_similarity']['average_score'],
        'phenodigm_score': r['phenodigm_score']['average_score'],
        'phenodigm_score_omim_self': r['omim_self_phenodigm_score']['average_score'],
        'phenodigm_score_orphanet_self': r['orphanet_self_phenodigm_score']['average_score'],
    } for r in results])

In [7]:
score_df

,disease,ancestor_information_content,ancestor_information_content_omim_self,ancestor_information_content_orphanet_self,jaccard_similarity,jaccard_similarity_omim_self,jaccard_similarity_orphanet_self,phenodigm_score,phenodigm_score_omim_self,phenodigm_score_orphanet_self
0,MONDO:0008843,13.523721,16.350819,15.337696,0.768806,1.0,1.0,3.183950,4.035342,3.909450
1,MONDO:0012658,12.993848,15.466155,15.945577,0.749854,1.0,1.0,3.020470,3.929436,3.985654
2,MONDO:0007215,12.763739,17.020723,15.844160,0.722589,1.0,1.0,2.977500,4.120375,3.975599
3,MONDO:0012072,14.174976,16.578860,16.452107,0.767127,1.0,1.0,3.250201,4.060397,4.053891
4,MONDO:0010392,13.799336,15.651929,15.441478,0.930575,1.0,1.0,3.565088,3.946377,3.915449
...,...,...,...,...,...,...,...,...,...,...
2092,MONDO:0013737,14.584981,16.366311,15.814934,0.869527,1.0,1.0,3.507886,4.040150,3.967945
2093,MONDO:0013400,13.380031,16.584899,16.951649,0.723024,1.0,1.0,3.057376,4.064994,4.110144
2094,MONDO:0008087,12.853699,16.438208,15.836758,0.773554,1.0,1.0,3.086015,4.044986,3.972138
2095,MONDO:0011035,12.761761,17.142761,15.570460,0.756597,1.0,1.0,3.059558,4.135477,3.922544


Export the full results as JSON, export the score dataframe as tsv

In [8]:
import json 

# write results as json
with open('output/phenotype_profile_comparison_results.json', 'w') as f:
    json.dump(results, f)

# write results as tsv
score_df.to_csv('output/phenotype_profile_comparison_scores.tsv', sep='\t', index=False)